#### SQL info

In [ ]:
window function and group by are not used in the same query, can be used with cte. like define a cte and perfrom group by operation and the use that cte table to perfom window function operation.

#### [Q3-Premium Customers](https://100daysofsql.com/new-courses/1/coding-test/4)

In [ ]:
-- Namastekart, an e-commerce company want to start special reward program for their premium customers.  The customers who have placed more number of orders than the average number of orders placed by each customers are considered as premium customers.

-- Write an SQL to find the list of premium customers along with the number of orders placed by each of them. 

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- customer_name	varchar(20)
-- sales	int
 

-- Example Input

-- orders

-- order_id	order_date	customer_name	sales
-- 1	2023-01-01	Alexa	1239
-- 2	2023-01-02	Alexa	1239
-- 3	2023-01-03	Alexa	1239
-- 4	2023-01-03	Alexa	1239
-- 5	2023-01-01	Ramesh	1239
-- 6	2023-01-02	Ramesh	1239
-- 7	2023-01-03	Ramesh	1239
-- 8	2023-01-03	Neha	1200
-- 9	2023-01-03	Subhash	100
-- 10	2023-01-03	Subhash	230
 

-- Expected Output:

-- customer_name	no_of_orders
-- Alexa	4
-- Ramesh	3
-- Explanation: There are total 10 orders placed by 4 customers so the average number order placed by each customers is 10/4 = 2.5 . Alexa and ramesh have done more than 2.5 orders so they are in the output. 

In [ ]:
-- #my_sol

with cte as 
(select count(*)/count(distinct(customer_name)) as avg_orders from orders)

-- select * from cte
select customer_name, count(*) as no_of_orders
from orders, cte
group by customer_name
having count(*)>cte.avg_orders

#provided_sol

select customer_name,count(*) as no_of_orders 
from orders
group by customer_name having count(*)>(select count(*)*1.0/count(distinct customer_name) as avg_no_of_orders  from orders)

**chat gpt**

In SQL, when you use a comma between two tables in the `FROM` clause, it is creating a Cartesian product (cross join) between those tables. This means that every row from the first table is paired with every row from the second table. In your query:

```sql
select customer_name, count(*) as no_of_orders
from orders, cte
group by customer_name
having count(*) > cte.avg_orders
```

The `orders` table and the common table expression (CTE) `cte` are combined using a cross join. Since `cte` is expected to return only one row (it contains an aggregate calculation), the result is effectively adding the `avg_orders` value to every row from the `orders` table. 

Here's a step-by-step breakdown of the query:

1. The CTE `cte` is defined, which calculates the average number of orders per customer:
   ```sql
   with cte as 
   (select count(*)/count(distinct(customer_name)) as avg_orders from orders)
   ```
   
2. The main query:
   ```sql
   select customer_name, count(*) as no_of_orders
   from orders, cte
   group by customer_name
   having count(*) > cte.avg_orders
   ```
   - `from orders, cte`: This creates a Cartesian product of `orders` and the single row from `cte`. Because `cte` returns a single row, each row of `orders` is effectively augmented with the `avg_orders` value.
   - `group by customer_name`: This groups the result by `customer_name`.
   - `having count(*) > cte.avg_orders`: This filters the groups to only include customers whose number of orders is greater than the average number of orders per customer.

So, while a join is not explicitly specified, the cross join ensures that each row of `orders` has access to the `avg_orders` value from the CTE. This allows the `having` clause to compare the count of orders for each customer against this average.

If the CTE were to return more than one row, this query would not work correctly because the Cartesian product would multiply the rows from `orders` by the number of rows in the CTE, leading to incorrect results. In this case, since the CTE is designed to return exactly one row, the query functions as intended.

#### [4 - LinkedIn Top Voice](https://100daysofsql.com/new-courses/1/coding-test/5)

In [ ]:
-- LinkedIn is a professional social networking app. They want to give top voice badge to their best creators to encourage them to create more quality content. A creator qualifies for the badge if he/she satisfies following criteria. 

-- 1- Creator should have more then 50k followers.

-- 2- Creator should have more than 100k impressions on the posts that they published in the month of Dec-2023.

-- 3- Creator should have published atleast 3 posts in Dec-2023.

-- Write a SQL to get the list of top voice creators name along with no of posts and impressions by them in the month of Dec-2023.

-- Table: creators(primary key : creator_id)

-- column name	data type 
-- creator_id 	int
-- creator_name 	varchar(20)
-- followers 	int

-- Table: posts(primary key : post_id)

-- column name	data type 
-- creator_id 	int
-- post_id 	varchar(3)
-- publish_date 	date
-- impressions 	int
 

-- Example output:

-- creator_name	no_of_posts	 total_impressions
-- Ankit Bansal	3	132000
 

In [ ]:
-- my_sol

select c.creator_name, count(*) as no_of_posts, sum(p.impressions) as total_impressions
from creators c join posts p on c.creator_id=p.creator_id
where c.followers>50000 and 
strftime('%Y%m', p.publish_date) = '202312'
group by c.creator_name
having sum(impressions)>100000 and 
count(*)>=3

-- site_sol
select c.creator_name,count(p.post_id) as no_of_posts, sum(p.impressions) as total_impressions
from creators c
inner join posts p on c.creator_id=p.creator_id
where c.followers>50000 and strftime('%Y%m', p.publish_date)='202312'
group by c.creator_name,c.followers
having sum(p.impressions)>100000 and count(p.post_id)>=3 

#### [5 - CIBIL Score](https://100daysofsql.com/new-courses/1/coding-test/6)

CIBIL score, often referred to as a credit score, is a numerical representation of an individual's creditworthiness.While the exact formula used by credit bureaus like CIBIL may not be publicly disclosed and can vary slightly between bureaus, the following are some common factors that typically influence the calculation of a credit score:

Payment History: This accounts for the largest portion of your credit score. It includes factors such as whether you pay your bills on time, any late payments, defaults, bankruptcies, etc. Assume this accounts for 70 percent of your credit score.

Credit Utilization Ratio: This is the ratio of your credit card balances to your credit limits. Keeping this ratio low (ideally below 30%) indicates responsible credit usage. Assume it accounts for 30% of your score and below logic to calculate it: 

           Utilization below 30% = 1

           Utilization between 30% and 50% = 0.7

           Utilization above 50% = 0.5

Assume that we have credit card bills data for March 2023 based on that we need to calculate credit utilization ratio.

 

Final Credit score formula = (on_time_loan_or_bill_payment)/total_bills_and_loans * 70 + Credit Utilization Ratio * 30 

 

Table: customers(primary key : customer_id )

column name	data type 
customer_id 	int
credit_limit 	int
 

Table: loans(primary key : loan_id )

column name	data type 
customer_id 	int
loan_id 	int
loan_due_date 	date
 

Table: credit_card_bills(primary key : bill_id )

column name	data type 
customer_id 	int
bill_id 	int
bill_due_date 	date
balance_amount 	int
 

Table: customer_transactions(primary key : loan_bill_id )

column name	data type 
loan_bill_id 	int
transaction_type 	varchar(10)
transaction_date	date
 

Expected Output: 

customer_id	cibil_score
1	82.5
2	91.0

#### [11 - New and Repeat Customers](https://100daysofsql.com/new-courses/1/coding-test/12)

In [ ]:
NamasteKart an ecommerce company wants to build a very imporant business metrics where they want to track on daily basis how many new and repeat customers are purchasing products from their website. A new customer is defined when he purchased anything for the first time from the website and repeat customer is someone who has done atleast one purchase in the past.

Table: customer_orders (primary key : order_id)

column name	datatype
order_id	int
order_date	date
customer_id 	int
order_amount 	int

Expected Output:

order_date	new_customers	repeat_customers
2022-01-01	3	0
2022-01-02	2	1
2022-01-03	1	2
 

In [ ]:
-- Create the customer_orders table
CREATE TABLE customer_orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT,
    order_amount INT
);

-- Insert data into the customer_orders table
INSERT INTO customer_orders (order_id, customer_id, order_date, order_amount) VALUES
(1, 100, '2022-01-01', 2000),
(2, 200, '2022-01-01', 2500),
(3, 300, '2022-01-01', 2100),
(4, 100, '2022-01-02', 2000),
(5, 400, '2022-01-02', 2200),
(6, 500, '2022-01-02', 2700),
(7, 100, '2022-01-03', 3000),
(8, 400, '2022-01-03', 1000),
(9, 600, '2022-01-03', 3000);


In [ ]:
with cte as
(select * ,
min(order_date) over(partition by customer_id ) as first_order
from customer_orders
)

select 
order_date,
sum(case when order_date=first_order then 1 else 0 end) as new_customers,
sum(case when order_date!=first_order then 1 else 0 end) as repeat_customers
from cte
group by order_date

In [ ]:
-- # provided solution

with first_order_date as 
(select customer_id,min(order_date) as first_order 
from customer_orders 
group by customer_id
)
select co.order_date 
,sum(case when co.order_date=fod.first_order then 1 else 0 end) as new_customers
,sum(case when co.order_date>fod.first_order then 1 else 0 end) as repeat_customers
from customer_orders co 
inner join first_order_date fod on co.customer_id=fod.customer_id
group by co.order_date 

#### [12 - The Little Master](https://100daysofsql.com/new-courses/1/coding-test/13)

In [ ]:
-- Sachin Tendulkar - Also known as little master. You are given runs scored by Sachin in his first 10 matches. You need to write an SQL to get match number when he completed 500 runs and his batting average at the end of 10 matches.

-- Batting Average = (Total runs scored) / (no of times batsman got out)

-- Round the result to 2 decimal places.

-- Table: sachin(primary key : match_no)

-- column name	datatype
-- match_no	int
-- runs_scored	int
-- status	varchar(10)
 
-- Expected Output:

-- match_no	batting_average
-- 8	70.33

In [ ]:
-- Create sachin table
CREATE TABLE sachin (
    match_no INT PRIMARY KEY,
    runs_scored INT,
    status VARCHAR(10)
);

-- Insert data into sachin table
INSERT INTO sachin (match_no, runs_scored, status) VALUES
(1, 53, 'out'),
(2, 59, 'not-out'),
(3, 113, 'out'),
(4, 29, 'out'),
(5, 0, 'out'),
(6, 39, 'out'),
(7, 73, 'out'),
(8, 149, 'out'),
(9, 93, 'out'),
(10, 25, 'out');


In [ ]:
-- my sol

with batting_average as
(
    select round(sum(runs_scored)*1.0/sum(case when status='out' then 1 else 0 end),2) as batting_average from sachin

)
, running_score as
(
    select *, 
    sum(runs_scored) over(rows between unbounded preceding and current row) as running_score
    from sachin, batting_average
)
select match_no, batting_average
from running_score
where running_score>=500
limit 1

In [ ]:
-- provided sol

with cte as (
select *
,sum(runs_scored) over()*1.0 / sum(case when status='out' then 1 else 0 end) over() as batting_average
,sum(runs_scored) over(order by match_no rows between unbounded preceding and current row) as running_sum
from sachin
)
select match_no,round(batting_average,2) as batting_average
from (select *, row_number() over(order by running_sum) as rn
from cte where running_sum>500)
where rn=1;

#### [13 - Trending Products](https://100daysofsql.com/new-courses/1/coding-test/14)

In [ ]:
-- code to insert the orders table

-- Step 1: Create the table
CREATE TABLE orders (
    order_month DATE,
    product_id VARCHAR(5),
    sales INT,
    PRIMARY KEY (order_month, product_id)
);


-- Step 2: Insert the data
INSERT INTO orders (order_month, product_id, sales) VALUES
('2023-01-01', 'p1', 100),
('2023-01-01', 'p2', 500),
('2023-02-01', 'p1', 700),
('2023-02-01', 'p2', 300),
('2023-03-01', 'p1', 900),
('2023-03-01', 'p2', 700),
('2023-04-01', 'p1', 2000),
('2023-04-01', 'p2', 1100);


In [ ]:
with cte as 
(select *,
lag(sales,1) over(partition by product_id order by order_month)  as prev1,
lag(sales,2) over(partition by product_id order by order_month) as prev2,
lag(sales,1) over(partition by product_id order by order_month)+lag(sales,2) over(partition by product_id order by order_month) as prev_total
from orders)

select order_month,product_id
from cte
where prev_total is not null and prev_total<sales;

In [ ]:
with cte2 as
(select *,
sum(sales) over(partition by product_id order by order_month rows between 2 preceding and 1 preceding) as prev_total

from orders)

select 
order_month, product_id

from cte2
where sales>prev_total and order_month not in (select distinct order_month from orders order by order_month asc limit 2)


In [ ]:
-- provided solution
with cte as (
select * 
,sum(sales) over(partition by product_id order by order_month rows between 2 preceding and 1 preceding) as last2
,row_number() over(partition by product_id order by order_month) as rn
from orders 
)
select order_month,product_id from cte 
where rn>=3 and sales > last2

#### [14 - Uber Rides](https://100daysofsql.com/new-courses/1/coding-test/15)

In [ ]:
code to create the table 

-- Create the drivers table
CREATE TABLE drivers (
    id VARCHAR(10),
    start_time TIME,
    end_time TIME,
    start_loc VARCHAR(1),
    end_loc VARCHAR(1),
    PRIMARY KEY (id, start_time)
);

-- Insert data into the drivers table
INSERT INTO drivers (id, start_time, end_time, start_loc, end_loc) VALUES
('dri_1', '09:00:00', '09:30:00', 'a', 'b'),
('dri_1', '09:30:00', '10:30:00', 'b', 'c'),
('dri_1', '11:00:00', '11:30:00', 'd', 'e'),
('dri_1', '12:00:00', '12:30:00', 'f', 'g'),
('dri_1', '13:30:00', '14:30:00', 'c', 'h'),
('dri_2', '12:15:00', '12:30:00', 'f', 'g'),
('dri_2', '13:30:00', '14:30:00', 'c', 'h');


In [ ]:
with cte as
(select *,
lead(start_time,1) over(partition by id order by start_time) as nxt_ride,
lead(start_loc,1) over(partition by id order by start_time) as nxt_start

from drivers)

select id, count(*) as total_rides,
sum(case when end_time=nxt_ride and end_loc=nxt_start then 1 else 0 end) as profit_rides
from cte
group by id


In [ ]:
-- provided solution

with cte as (
select * 
, lag(end_time,1) over(partition by id order by start_time) as prev_end_time
, lag(end_loc,1) over(partition by id order by start_time) as prev_end_loc
from drivers
)
select id, count(*) as total_rides , 
sum(case when start_time = prev_end_time and start_loc=prev_end_loc then 1 else 0 end) as profit_rides
from cte 
group by id

#### [15 - The United States of America](https://100daysofsql.com/new-courses/1/coding-test/16)

In [ ]:
''' In some poorly designed UI applications, there's often a lack of data input restrictions. For instance, in a free text field for the country, users might input variations such as 'USA,' 'United States of America,' or 'US.'

Suppose we have survey data from individuals in the USA about their job satisfaction, rated on a scale of 1 to 5. Write a SQL query to count the number of respondents for each rating on the scale. Additionally, include the country name in the format that occurs most frequently in that scale.

Table: survey (primary key : name)

column name	datatype
name 	varchar(10)
job_satisfaction 	int
country 	varchar(20)

Expected output:

job_satisfaction | country | number_of_respondents
4	USA	3
5	United States	4

'''

In [ ]:
-- Create the survey table
CREATE TABLE survey (
    name VARCHAR(10) PRIMARY KEY,
    job_satisfaction INT,
    country VARCHAR(20)
);

-- Insert data into the survey table
INSERT INTO survey (name, job_satisfaction, country) VALUES
('Alex', 4, 'USA'),
('Saurabh', 5, 'US'),
('Mark', 4, 'United States'),
('Shane', 4, 'USA'),
('Kim', 5, 'United States'),
('Joe', 5, 'USA'),
('Mira', 5, 'United States');


In [ ]:
with cte as (
select country,job_satisfaction,count(*) as cnt 
from survey
group by country,job_satisfaction
)
, cte2 as (
select *, sum(cnt) over(partition by job_satisfaction) as number_of_respondents
,max(cnt) over(partition by job_satisfaction) as max_cnt
from cte
)
select job_satisfaction,country,number_of_respondents from cte2 
where cnt=max_cnt


In [ ]:
-- my_sol
-- select  * from survey order by job_satisfaction;

with responses as
(
    select job_satisfaction, count(*) as number_of_respondents
    from survey
    group by job_satisfaction
)
, countries as
(
    select job_satisfaction, country, count(*) as country_count
    from survey
    group by job_satisfaction, country

)
, countries_rank as
(
    select job_satisfaction, country, country_count,
    dense_rank() over(partition by job_satisfaction order by country_count desc) as dr
    from countries
)
-- select * from countries_rank

select r.job_satisfaction, cr.country, number_of_respondents
from responses r join countries_rank cr on r.job_satisfaction=cr.job_satisfaction
where dr=1


#### [16 - Product Recommendation](https://100daysofsql.com/new-courses/1/coding-test/17)

In [ ]:
'''
Product recommendation. Just the basic type (“customers who bought this also bought…”). That, in its simplest form, is an outcome of basket analysis. Write a SQL to find the product pairs along with the count of times they have been purchased together. Based on the history ecommerce website can recommend products to new user.

Table: orders (primary key : order_id)

column name	datatype
order_id	int
customer_id 	int
product_id 	varchar(2)

Expected Output:

product_1	product_2	purchase_frequency
p2	p1	2
p3	p1	1
p3	p2	1
p4	p1	1
p4	p2	1
Note : In the output product_1 values should be greater than product_2 values. eg: pair p1,p2 or p2,p1 represent same thing but we need to show p2,p1 in the output.

'''

In [ ]:
CREATE TABLE orders (
    order_id INT NOT NULL,
    customer_id INT NOT NULL,
    product_id VARCHAR(2) NOT NULL,
    PRIMARY KEY (order_id, product_id)
);

INSERT INTO orders (order_id, customer_id, product_id) VALUES
(1, 1, 'p1'),
(1, 1, 'p2'),
(1, 1, 'p3'),
(2, 2, 'p1'),
(2, 2, 'p2'),
(2, 2, 'p4'),
(3, 1, 'p5');

In [ ]:
-- my sol
select 
o1.product_id, o2.product_id, count(*) as purchase_frequency
from orders o1 cross join orders o2
on o1.order_id=o2.order_id and o1.product_id!=o2.product_id and o1.product_id>o2.product_id
group by o1.product_id, o2.product_id


In [ ]:
select 
    o1.product_id as product_1, o2.product_id as product_2,
    count(*) as purchase_frequency
from 
    orders o1 
    join orders o2 on o1.order_id=o2.order_id
where 
    o1.product_id>o2.product_id
group by 
    o1.product_id, o2.product_id

#### [17 - Account Balance](https://100daysofsql.com/new-courses/1/coding-test/18)

In [ ]:
-- You are given a list of users and their opening account balance along with the trasaction done by them. Write a SQL to calculate their account balance at the end of all the transactions. Please note that users can do transactions among themselves as well. 

-- Table: users(primary key : user_id)

-- column name	datatype
-- user_id 	int
-- username 	varchar(10)
-- opening_balance 	int
 

-- Table: transactions (primary key : id)

-- column name	datatype
-- id	int
-- from_userid 	int
-- to_userid  	int
-- amount 	int

-- Expected Output:

-- user_id	final_balance
-- Ankit	2000
-- Rahul	10200
-- Amit	2800
-- Agam	7500

In [ ]:
-- Creating the users table
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    username VARCHAR(10) NOT NULL,
    opening_balance INTEGER NOT NULL
);

-- Creating the transactions table
CREATE TABLE transactions (
    id INTEGER PRIMARY KEY,
    from_userid INTEGER NOT NULL,
    to_userid INTEGER NOT NULL,
    amount INTEGER NOT NULL,
    FOREIGN KEY (from_userid) REFERENCES users(user_id),
    FOREIGN KEY (to_userid) REFERENCES users(user_id)
);

-- Inserting data into users table
INSERT INTO users (user_id, username, opening_balance) VALUES
(100, 'Ankit', 1000),
(101, 'Rahul', 9000),
(102, 'Amit', 5000),
(103, 'Agam', 7500);

-- Inserting data into transactions table
INSERT INTO transactions (id, from_userid, to_userid, amount) VALUES
(1, 100, 102, 500),
(2, 102, 101, 700),
(3, 101, 102, 600),
(4, 102, 100, 1500),
(5, 102, 101, 800),
(6, 102, 101, 300);


In [ ]:
-- my solution

with amt_sent as
(
    select from_userid, sum(amount) as sent
    from transactions 
    group by from_userid
)
, amt_received as
(
    select to_userid, sum(amount) as received
    from transactions 
    group by to_userid
)


select 
username as user_id, opening_balance-coalesce(sent,0)+coalesce(received,0) as final_balance

from users u left join amt_sent on u.user_id=amt_sent.from_userid 
left join amt_received on u.user_id=amt_received.to_userid

#### [18 - boAt Lifestyle](https://100daysofsql.com/new-courses/1/coding-test/19)

In [ ]:
-- boAt Lifestyle is focusing on influencer marketing to build and scale their brand. They want to partner with power creators for their upcoming campaigns. The creators should satisy below conditions to qualify :

-- 1- They should have 100k+ followers on atleast 2 social media platforms and

-- 2- They should have atleast 50k+ views on their latest YouTube video.

-- Write an SQL to get creator id and name satisfying above conditions.

-- Table: creators(primary key : order_id)

-- column name	datatype
-- id 	int
-- name 	varchar(10)
-- platform 	varchar(10)
-- followers 	int

-- Table: youtube_videos(primary key : order_id)

-- column name	datatype
-- id 	int
-- creator_id 	int
-- publish_date 	date
-- views	int
 

-- Expected Output:

-- id	name
-- 102	Dhruv

In [ ]:
-- Creating the creators table
CREATE TABLE creators (
    id INTEGER NOT NULL,
    name VARCHAR(10) NOT NULL,
    platform VARCHAR(10) NOT NULL,
    followers INTEGER NOT NULL,
    PRIMARY KEY (id, platform)
);

-- Creating the youtube_videos table
CREATE TABLE youtube_videos (
    id INTEGER PRIMARY KEY,
    creator_id INTEGER NOT NULL,
    publish_date DATE NOT NULL,
    views INTEGER NOT NULL,
    FOREIGN KEY (creator_id) REFERENCES creators(id)
);

-- Inserting data into creators table
INSERT INTO creators (id, name, platform, followers) VALUES
(100, 'Ankit', 'YouTube', 90000),
(100, 'Ankit', 'LinkedIn', 150000),
(101, 'Warikoo', 'YouTube', 500000),
(101, 'Warikoo', 'LinkedIn', 600000),
(101, 'Warikoo', 'Instagram', 800000),
(102, 'Dhruv', 'LinkedIn', 60000),
(102, 'Dhruv', 'YouTube', 900000),
(102, 'Dhruv', 'Instagram', 800000);

-- Inserting data into youtube_videos table
INSERT INTO youtube_videos (id, creator_id, publish_date, views) VALUES
(1, 100, '2024-01-01', 52000),
(2, 100, '2024-01-06', 62000),
(3, 101, '2024-01-05', 59000),
(4, 101, '2024-01-07', 22000),
(5, 102, '2024-01-05', 70000),
(6, 102, '2024-01-09', 90000);


#### [19 - Dynamic Pricing](https://100daysofsql.com/new-courses/1/coding-test/20)

In [ ]:
-- You are given a products table where a new row is inserted every time the price of a product changes. Additionally, there is a transaction table containing details such as order_date and product_ID for each order. Write an SQL query to calculate the total sales value for each product, considering the cost of the product at the time of the order date.

-- Table: products(primary key : product_id)

-- column name	datatype
-- product_id	int
-- price_date	date
-- price	int

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- product_id	int

-- Expected output:

-- product_id	total_sales
-- 100	510
-- 101	3450

In [ ]:
-- Creating the products table
CREATE TABLE products (
    product_id INTEGER NOT NULL,
    price_date DATE NOT NULL,
    price INTEGER NOT NULL,
    PRIMARY KEY (product_id, price_date)
);

-- Creating the orders table
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    order_date DATE NOT NULL,
    product_id INTEGER NOT NULL,
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
-- Inserting data into products table
INSERT INTO products (product_id, price_date, price) VALUES
(100, '2024-01-01', 150),
(100, '2024-01-21', 170),
(100, '2024-02-01', 190),
(101, '2024-01-01', 1000),
(101, '2024-01-27', 1200),
(101, '2024-02-05', 1250);

-- Inserting data into orders table
INSERT INTO orders (order_id, order_date, product_id) VALUES
(1, '2024-01-05', 100),
(2, '2024-01-21', 100),
(3, '2024-02-20', 100),
(4, '2024-01-07', 101),
(5, '2024-02-04', 101),
(6, '2024-02-05', 101);


In [ ]:
with cte as
(
    select 
        *, julianday(order_date)-julianday(price_date) as days_diff
    from 
        orders o 
        left join products p on o.order_date>=p.price_date and o.product_id=p.product_id
)
, cte2 as 
(
    select 
        *, dense_rank() over(partition by product_id, order_date order by days_diff asc) as dr
    from 
        cte
)
select 
    product_id, sum(price) as total_sales
from 
    cte2
where 
    dr=1
group by 
    product_id
-- my soln

In [ ]:
with price_range as (
select *
, date(lead(price_date,1,'9999-12-31') over(partition by product_id order by price_date),'-1 days') as price_end_date
 from products
)
select p.product_id , sum(p.price) as total_sales
from orders o 
inner join price_range p on o.product_id=p.product_id 
and o.order_date between p.price_date and p.price_end_date
group by p.product_id;
-- provided soln

#### [20 - Income Tax Returns](https://100daysofsql.com/new-courses/1/coding-test/21)

In [ ]:
-- Income tax returns are supposed to file every year before due date announce by govt every year . Goverment wants to find out people who have filed late returns or people who have completely skipped filing returns. 

-- Write an SQL to list all the users along with the year for which they have done late filing or missed filing the returns. Create a comment column as well specifying if its a 'late return' or  'missed'.

-- Table: income_tax_dates(primary key : financial_year)

-- column name	datatype
-- financial_year 	varchar(4)
-- file_start_date 	date
-- file_due_date 	date
 

-- Table: users (primary key : user_id,financial_year)

-- column name	datatype
-- user_id 	int
-- financial_year 	varchar(4)
-- return_file_date 	date
 


-- Expected Output:

-- user_id	financial_year	comment
-- 1	FY21	late return
-- 1	FY22	missed
-- 2	FY23	late return
 

In [ ]:
-- Creating the income_tax_dates table
CREATE TABLE income_tax_dates (
    financial_year VARCHAR(4) PRIMARY KEY,
    file_start_date DATE NOT NULL,
    file_due_date DATE NOT NULL
);

-- Creating the users table
CREATE TABLE users (
    user_id INTEGER NOT NULL,
    financial_year VARCHAR(4) NOT NULL,
    return_file_date DATE NOT NULL,
    PRIMARY KEY (user_id, financial_year),
    FOREIGN KEY (financial_year) REFERENCES income_tax_dates(financial_year)
);
-- Inserting data into income_tax_dates table
INSERT INTO income_tax_dates (financial_year, file_start_date, file_due_date) VALUES
('FY20', '2020-05-01', '2020-08-31'),
('FY21', '2021-06-01', '2021-09-30'),  -- Changed '2021-09-31' to '2021-09-30' to correct the date
('FY22', '2022-05-05', '2022-08-29'),
('FY23', '2023-05-05', '2023-08-31');

-- Inserting data into users table
INSERT INTO users (user_id, financial_year, return_file_date) VALUES
(1, 'FY20', '2020-05-10'),
(1, 'FY21', '2021-10-10'),
(1, 'FY23', '2023-08-20'),
(2, 'FY20', '2020-05-15'),
(2, 'FY21', '2021-09-10'),
(2, 'FY22', '2022-08-20'),
(2, 'FY23', '2023-10-10');


In [ ]:
-- my soln
with all_users_years as 
(
select 
u.user_id,it.financial_year,it.file_due_date

from users u , income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)

select * from 
(select  -- *
a.user_id, a.financial_year, 
case when u.return_file_date>a.file_due_date then 'late return'
when u.return_file_date<a.file_due_date then 'on_time'
else 'missed' end as comment
from all_users_years a left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year) a
where comment !='on_time'

In [ ]:
-- provided solution
with all_users_years as 
(
select u.user_id,it.financial_year,it.file_due_date
from users u cross join income_tax_dates it
group by u.user_id,it.financial_year,it.file_due_date
)
select a.user_id,a.financial_year
,case when u.return_file_date > a.file_due_date then 'late return' else 'missed' end as comment
from all_users_years a 
left join users u on a.user_id=u.user_id and a.financial_year=u.financial_year 
where u.return_file_date > a.file_due_date or u.return_file_date is null;

#### [21 - Malware Detection](https://100daysofsql.com/new-courses/1/coding-test/22)

In [ ]:
-- There are multiple antivirus softwares which are running on the systems and you have the data of how many malware they have detected  in each run.  You need to find out how many malware each software has detected in their latest run and what is the difference between the number of malwares detected in latest run and the second last run for each software. 

-- Please note that list only the softwares which have run for atleast 2 times and have detected atleast 10 malware in the latest run.

-- Table: malware(primary key : software_id,run_date)

-- column name	datatype
-- software_id	int
-- run_date	datetime
-- malware_detected	int

-- Expected output:
-- software_id	latest_run_count	difference_to_previous
-- 101	12	2

In [ ]:
CREATE TABLE malware (
    software_id INT PRIMARY KEY,
    run_date DATETIME,
    malware_detected INT
);
INSERT INTO malware (software_id, run_date, malware_detected) VALUES
(100, '2024-01-01 02:00:01', 12),
(100, '2024-01-01 03:12:01', 15),
(100, '2024-01-01 16:00:01', 9),
(101, '2024-01-01 12:00:00', 9),
(101, '2024-01-01 16:00:00', 10),
(101, '2024-01-01 18:00:00', 12),
(102, '2024-01-01 12:00:00', 14);


In [ ]:
-- my solution1
with runs as
(   select * , count(*) as run_count
    from malware
    group by software_id
    having count(*)>=2
    )

, runs_rank as
(
    select * ,
    dense_rank() over(partition by software_id order by run_date desc) as dr,
    lead(malware_detected) over(partition by software_id order by run_date desc) as leading
    from malware
)

select rc.software_id, rc.malware_detected as latest_run_count, 
rc.malware_detected-leading as difference_to_previous

from runs r left join runs_rank rc on r.software_id=rc.software_id
where dr=1 and rc.malware_detected>=10;

In [ ]:
-- my solution2
with cte as
(select *,
lag(run_date,1) over(partition by software_id order by run_date) as lagging_date,
rank() over(partition by software_id order by run_date desc) as rnk
from malware)

-- select * from cte
select cte.software_id, cte.malware_detected as latest_run_count,
cte.malware_detected-m.malware_detected  as difference_to_previous

from cte join malware m on cte.lagging_date=m.run_date
where cte.malware_detected >=10 and cte.rnk=1


In [ ]:
-- provided solution
with cte1 as (
select *,
row_number() over(partition by software_id order by run_date desc) as rn
 from malware 
)
,cte2 as (
select software_id
, sum(case when rn=1 then malware_detected end) as latest_run_count
, sum(case when rn=2 then malware_detected end) as previous_run
from cte1
where rn in (1,2)
group by software_id
having count(*)=2
)
select software_id,latest_run_count
,(latest_run_count-previous_run) as difference_to_previous 
from cte2
where latest_run_count>=10

#### [22 - Highly Paid Employees](https://100daysofsql.com/new-courses/1/coding-test/23)

In [ ]:
-- You are given the data of employees along with their salary and department . Write an SQL to find list of employees who has salary greater than average employee salary of the company.  However , While calculating the company average salary to compare with an employee salary do not consider salaries of that employee's department.

 

-- Table: employee(primary key : emp_id)

-- column	datatype
-- emp_id	int
-- salary	date
-- department	varchar(20)
 


-- emp_id	salary	department
-- 102	50000	Analytics
-- 103	45000	Engineering
-- 104	48000	Engineering
-- 105	51000	Engineering
-- 106	46000	Science

In [ ]:
CREATE TABLE employee (
    emp_id INT PRIMARY KEY,
    salary INT,
    department VARCHAR(20)
);
INSERT INTO employee (emp_id, salary, department) VALUES
(100, 40000, 'Analytics'),
(101, 30000, 'Analytics'),
(102, 50000, 'Analytics'),
(103, 45000, 'Engineering'),
(104, 48000, 'Engineering'),
(105, 51000, 'Engineering'),
(106, 46000, 'Science'),
(107, 38000, 'Science'),
(108, 37000, 'Science');


In [ ]:
select * from employee e1
where salary>(select avg(salary) from employee e2 where e1.department!=e2.department)

#### [23 - Warikoo 20-6-20](https://100daysofsql.com/new-courses/1/coding-test/24)

In [ ]:
-- Ankur Warikoo, an influential figure in Indian social media, shares a guideline in one of his videos called the 20-6-20 rule for determining whether one can afford to buy a phone. The rule entails three conditions:

-- Having enough savings to cover a 20 percent down payment.
-- Utilizing a maximum 6-month EMI plan (no-cost) for the remaining cost.
-- Ensuring that the monthly EMI does not exceed 20 percent of one's monthly salary.
-- Given the salary and savings of various users, along with data on phone costs, the task is to write an SQL to generate a list of phones (comma-separated) that each user can afford based on these criteria.

-- Table: users (primary key : user_name)

-- column name	datatype
-- user_name 	varchar(20)
-- monthly_salary 	int
-- savings 	int

-- Table: phones (primary key : phone_name)

-- column name	datatype
-- phone_name 	varchar(15)
-- cost 	int
 

-- Expected Output:

-- user_name	affordable_phones
-- Rahul	iphone-12,oneplus-12
-- Vivek	oneplus-12

In [ ]:
-- Create users table
CREATE TABLE users (
    user_name VARCHAR(20) PRIMARY KEY,
    monthly_salary INT,
    savings INT
);

-- Insert data into users table
INSERT INTO users (user_name, monthly_salary, savings) VALUES
('Rahul', 40000, 15000),
('Vivek', 70000, 10000);

-- Create phones table
CREATE TABLE phones (
    phone_name VARCHAR(15) PRIMARY KEY,
    cost INT
);

-- Insert data into phones table
INSERT INTO phones (phone_name, cost) VALUES
('iphone-12', 60000),
('oneplus-12', 50000),
('iphone-14', 70000);


In [ ]:
select user_name, group_concat(phone_name,",")  as affordable_phones
from phones, users
where savings>=cost*0.20  and monthly_salary*0.20>=cost*0.8/6
group by user_name;

#### [31 - Driver Ratings](https://100daysofsql.com/new-courses/1/coding-test/32)

In [ ]:
-- Suppose you are a data analyst working for a company that operates a ride-sharing platform similar to Uber. Your company is interested in analyzing the performance of drivers based on their ratings and wants to categorize them into different performance tiers. You are given a table named driver_ratings containing the following columns:

-- Table : driver_ratings (primary key : driver_id)

-- column name	datatype
-- driver_id	int
-- avg_rating	decimal(3,2)
 

-- Your task is to write an SQL query to categorize drivers equally into three performance tiers (top, middle, and bottom) based on their average ratings. Drivers with the highest average ratings should be placed in the top tier, drivers with ratings below the top tier but above the bottom tier should be placed in the middle tier, and drivers with the lowest average ratings should be placed in the bottom tier.

-- Expected Output:

-- driver_id	avg_rating	performance_tier
-- 7	4.9	Top
-- 1	4.8	Top
-- 5	4.7	Top
-- 12	4.6	Top
-- 2	4.5	Middle
-- 9	4.4	Middle
-- 4	4.2	Middle
-- 11	4.1	Middle
-- 3	3.9	Bottom
-- 8	3.8	Bottom
-- 6	3.6	Bottom
-- 10	3.5	Bottom
 

In [ ]:
-- Create the driver_ratings table
CREATE TABLE driver_ratings (
    driver_id INT PRIMARY KEY,
    avg_rating DECIMAL(3,2)
);

-- Insert data into the driver_ratings table
INSERT INTO driver_ratings (driver_id, avg_rating) VALUES
(1, 4.80),
(2, 4.50),
(3, 3.90),
(4, 4.20),
(5, 4.70),
(6, 3.60),
(7, 4.90),
(8, 3.80),
(9, 4.40),
(10, 3.50),
(11, 4.10),
(12, 4.60);


In [ ]:
-- my sol
with cte2 as 
(select driver_id, row_number() over(order by avg_rating desc) as rankings 
from driver_ratings)


select 
dr.driver_id, avg_rating,
case when rankings*1.0/(select count(*)/3 from driver_ratings) <=1 then 'Top'
    when rankings*1.0/(select count(*)/3 from driver_ratings) <=2 then 'Middle'
    when rankings*1.0/(select count(*)/3 from driver_ratings) <=3 then 'Bottom' end as performance_tier
 from driver_ratings dr join cte2 on dr.driver_id=cte2.driver_id
order by avg_rating desc


In [ ]:
--  provided sol
SELECT driver_id,avg_rating,
    CASE
        WHEN tier = 1 THEN 'Top'
        WHEN tier = 2 THEN 'Middle'
        ELSE 'Bottom'
    END AS performance_tier
FROM (
    SELECT
        driver_id,avg_rating
        ,NTILE(3) OVER (ORDER BY avg_rating DESC) AS tier
    FROM
        driver_ratings
) AS ranked_drivers;

#### [32 - Customer Behavior](https://100daysofsql.com/new-courses/1/coding-test/33)

In [ ]:
-- Suppose you are a data analyst working for Zomato (a online food delivery company)  . Zomato is interested in analyzing customer food ordering behavior and wants to identify customers who have exhibited inconsistent patterns over time.

-- Your task is to write an SQL query to identify customers who have placed orders on both weekdays and weekends, but with a significant difference in the average order amount between weekdays and weekends. Specifically, you need to identify customers who have a minimum of 3 orders placed both on weekdays and weekends each, and where the average order amount on weekends is at least 20% higher than the average order amount on weekdays.

-- Your query should return the customer_id, the average order amount on weekends, the average order amount on weekdayss, and the percentage difference in average order amount between weekends and weekdays for each customer meeting the criteria.

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- customer_id 	int
-- order_amount 	decimal(10,2)
 

-- Expected Output:

-- customer_id	weekend_avg_amount	weekday_avg_amount	percent_diff
-- 104	135.0	100.0	35.0
 

In [ ]:
-- Create the orders table
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT,
    order_amount DECIMAL(10,2)
);

-- Insert data into the orders table
INSERT INTO orders (order_id, customer_id, order_date, order_amount) VALUES
(1, 101, '2023-01-04', 100.00),
(2, 101, '2023-01-05', 200.00),
(3, 101, '2023-01-06', 80.00),
(4, 101, '2023-01-07', 110.00),
(5, 101, '2023-01-08', 90.00),
(6, 101, '2023-01-14', 130.00),
(7, 101, '2023-01-15', 150.00),
(8, 101, '2023-01-21', 100.00),
(9, 101, '2023-01-22', 120.00),
(10, 101, '2023-01-28', 90.00),
(11, 102, '2023-01-29', 110.00),
(12, 102, '2023-02-04', 80.00),
(13, 102, '2023-02-05', 100.00),
(14, 102, '2023-02-11', 120.00),
(15, 102, '2023-02-15', 140.00),
(16, 102, '2023-02-18', 80.00),
(17, 102, '2023-02-19', 100.00),
(18, 102, '2023-02-25', 120.00),
(19, 102, '2023-02-29', 140.00),
(20, 102, '2023-03-04', 80.00),
(31, 104, '2023-04-09', 140.00),
(32, 104, '2023-04-13', 80.00),
(33, 104, '2023-04-16', 100.00),
(34, 104, '2023-04-22', 200.00),
(35, 104, '2023-04-27', 140.00),
(36, 104, '2023-04-28', 80.00),
(37, 104, '2023-04-30', 100.00);


In [ ]:
-- my sol
with cte1 as 
(select *,
case when strftime('%w', order_date) in ('0','6') then 'weekend'
else 'weekday' end as day_type
from orders)

select customer_id
,round(sum(case when day_type='weekend' then order_amount else 0 end)*1.0/
sum(case when day_type='weekend' then 1 else 0 end)) as weekend_avg_amount

,round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
sum(case when day_type='weekday' then 1 else 0 end)) as weekday_avg_amount

,(round(sum(case when day_type='weekend' then order_amount else 0 end)*1.0/
sum(case when day_type='weekend' then 1 else 0 end))-round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
sum(case when day_type='weekday' then 1 else 0 end)))*100.0/
round(sum(case when day_type='weekday' then order_amount else 0 end)*1.0/
sum(case when day_type='weekday' then 1 else 0 end)) as percent_diff
from cte1 
group by customer_id
having sum(case when day_type='weekend' then 1 else 0 end)>=3 and
sum(case when day_type='weekday' then 1 else 0 end)>=3

order by percent_diff desc
limit 1


In [ ]:
-- provided sol

WITH order_summary AS (
    SELECT
        customer_id,
        CASE
            WHEN strftime('%w', order_date) IN ('0', '6') THEN 'Weekend'
            ELSE 'Weekday'
        END AS day_type,
        COUNT(*) AS order_count,
        AVG(order_amount) AS avg_order_amount
    FROM
        orders
    GROUP BY
        customer_id,
        day_type
),
weekend_weekday_summary AS (
    SELECT
        customer_id,
        MAX(CASE WHEN day_type = 'Weekend' THEN order_count END) AS weekend_order_count,
        MAX(CASE WHEN day_type = 'Weekday' THEN order_count END) AS weekday_order_count,
        MAX(CASE WHEN day_type = 'Weekend' THEN avg_order_amount END) AS weekend_avg_amount,
        MAX(CASE WHEN day_type = 'Weekday' THEN avg_order_amount END) AS weekday_avg_amount
    FROM
        order_summary
    GROUP BY
        customer_id
)
SELECT customer_id,weekend_avg_amount,weekday_avg_amount
,(weekend_avg_amount - weekday_avg_amount)*100 / weekday_avg_amount AS percent_diff
FROM weekend_weekday_summary
where weekend_avg_amount>1.2*weekday_avg_amount

#### [33 - Points Table](https://100daysofsql.com/new-courses/1/coding-test/34)

In [ ]:
-- You are given table of cricket match played in a ICC cricket tounamant with the details of winner for each match. You need to derive a points table using below rules.

-- 1- For each win a team gets 2 points. 

-- 2- For a loss team gets 0 points.

-- 3- In case of a draw both the team gets 1 points each.

-- Table: icc_world_cup 

-- column name	datatype
-- team_1 	varchar(20)
-- team_2 	varchar(20)
-- winner 	varchar(20)


-- Expected Output:

-- team_name	match_played	no_of_wins	no_of_losses	total_points
-- Aus	2	0	1	1
-- Eng	3	1	1	3
-- India	2	2	0	4
-- NZ	1	1	0	2
-- SA	2	0	1	1
-- SL	2	0	1	1

In [ ]:
-- Create the icc_world_cup table
CREATE TABLE icc_world_cup (
    team_1 VARCHAR(20),
    team_2 VARCHAR(20),
    winner VARCHAR(20)
);

-- Insert data into the icc_world_cup table
INSERT INTO icc_world_cup (team_1, team_2, winner) VALUES
('India', 'SL', 'India'),
('SL', 'Aus', 'Draw'),
('SA', 'Eng', 'Eng'),
('Eng', 'NZ', 'NZ'),
('Aus', 'India', 'India'),
('Eng', 'SA', 'Draw');


In [ ]:
-- my sol
with cte as
(
    select team_1 as team_name, 
    case when team_1 =winner then 1 else 0 end win_flag,
    case when winner = 'Draw' then 1 else 0 end draw_flag
    from icc_world_cup
    union all 
    select team_2 as team_name,
    case when team_2 =winner then 1 else 0 end win_flag,
    case when winner = 'Draw' then 1 else 0 end draw_flag
    from icc_world_cup
)

select team_name,
count(*) as match_played,
sum(win_flag) as no_of_wins, count(*)-sum(win_flag)-sum(draw_flag) as no_of_losses,
sum(win_flag)*2+sum(draw_flag) as total_points

from cte
group by team_name

In [ ]:
-- provided sol
with cte as (
select team_1 as team_name
, case when team_1=winner then 1 else 0 end as win_flag
, case when winner='Draw' then 1 else 0 end as draw_flag
from icc_world_cup 
union all
select team_2 as team_name
, case when team_2=winner then 1 else 0 end as win_flag
, case when winner='Draw' then 1 else 0 end as draw_flag
from icc_world_cup 
)
select team_name,count(*) as match_played
,sum(win_flag) as no_of_wins
,count(*)-sum(win_flag)-sum(draw_flag) as no_of_losses
,2*sum(win_flag)+sum(draw_flag) as total_points
from cte
group by team_name;

#### [34 - Customer Retention](https://100daysofsql.com/new-courses/1/coding-test/35)

In [ ]:
-- Customer retention can be defined as number of customers who continue to make purchases over a certain period compared to the total number of customers. Here's a step-by-step approach to calculate customer retention rate:

-- Determine the number of customers who made purchases in the current period (e.g., month : m )
-- Identify the number of customers from month m who made purchases in month m+1 , m+2 as well.
-- Suppose you are a data analyst working for an e-commerce company that sells various products online. The company is interested in measuring customer retention over the months to understand how many customers continue to make purchases over time. Your task is to write an SQL to derive customer retention month over month.

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	date
-- customer_id 	int


-- Expected Output:

-- current_month	future_month	total_customers	retained_customers
-- 202312	202401	3	2
-- 202312	202402	3	1
-- 202401	202402	3	2
-- Explanation: For first row , in 202312 there were 3 distinct customers and out of them 2 did purchase in 202401 so they are retained customers. Similarly second row says that out of 3 customers in 202312 only 1 customer did purchase in 202402 and so on.

In [ ]:
-- Create the orders table
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT
);

-- Insert data into the orders table
INSERT INTO orders (order_id, customer_id, order_date) VALUES
(1, 101, '2023-12-01'),
(2, 102, '2023-12-02'),
(3, 103, '2023-12-13'),
(4, 104, '2024-01-05'),
(5, 101, '2024-01-06'),
(6, 102, '2024-01-08'),
(7, 102, '2024-01-10'),
(8, 104, '2024-02-15'),
(9, 105, '2024-02-20'),
(10, 102, '2024-02-25');


In [ ]:
-- # my sol
with cte as (select 
distinct strftime('%Y%m', order_date) as yr_month , customer_id
from orders)

select 
    t1.yr_month as current_month, 
    count(distinct t1.customer_id) as total_customers,
    t2.yr_month as future_month, 
    count(distinct case when t1.customer_id= t2.customer_id then t1.customer_id end) as retained_customers
from 
    cte t1 join cte t2 on t1.yr_month<t2.yr_month
group by 
    t1.yr_month, t2.yr_month

In [ ]:
-- provided sol
with cte as (select 
distinct strftime('%Y%m', order_date) as yr_month , customer_id
from orders)

select 
    t1.yr_month as current_month, 
    count(distinct t1.customer_id) as total_customers,
    t2.yr_month as future_month, 
    count(distinct case when t1.customer_id= t2.customer_id then t1.customer_id end) as retained_customers
from 
    cte t1 join cte t2 on t1.yr_month<t2.yr_month
group by 
    t1.yr_month, t2.yr_month

#### [35 - Order Fulfillment](https://100daysofsql.com/new-courses/1/coding-test/36)

In [ ]:
-- You are given two tables: products and orders. The products table contains information about each product, including the product ID and available quantity in the warehouse. The orders table contains details about customer orders, including the order ID, product ID, order date, and quantity requested by the customer.

-- Write an SQL query to generate a report listing the orders that can be fulfilled based on the available inventory in the warehouse, following a first-come-first-serve approach based on the order date. Each row in the report should include the order ID, product name, quantity requested by the customer, quantity actually fulfilled, and a comments column as below:

-- If the order can be completely fulfilled then 'Full Order',If the order can be partially fullfilled then 'Partial Order',If order can not be fulfilled at all then 'No Order'.

 

-- Table: products(primary key : product_id)

-- column name	datatype
-- product_id 	int
-- product_name 	varchar(10)
-- available_quantity 	int
 


-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- product_id 	int
-- order_date	date
-- quantity_requested 	int
 


-- Expected Output:

-- order_id	product_name	quantity_requested	qty_fulfilled	comments
-- 1	Product A	5	5	Full Order
-- 2	Product A	7	5	Partial Order
-- 3	Product B	10	10	Full Order
-- 4	Product B	10	10	Full Order
-- 5	Product B	5	0	No Order
-- 6	Product C	4	4	Full Order
-- 7	Product C	5	5	Full Order
-- 8	Product D	4	4	Full Order
-- 9	Product D	5	5	Full Order
-- 10	Product D	8	1	Partial Order
-- 11	Product D	5	0	No Order


In [ ]:
-- Create the products table
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(10),
    available_quantity INT
);

-- Create the orders table
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    product_id INT,
    order_date DATE,
    quantity_requested INT,
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

-- Insert data into the products table
INSERT INTO products (product_id, product_name, available_quantity) VALUES
(1, 'Product A', 10),
(2, 'Product B', 20),
(3, 'Product C', 15),
(4, 'Product D', 10);

-- Insert data into the orders table
INSERT INTO orders (order_id, product_id, order_date, quantity_requested) VALUES
(1, 1, '2024-01-01', 5),
(2, 1, '2024-01-02', 7),
(3, 2, '2024-01-03', 10),
(4, 2, '2024-01-04', 10),
(5, 2, '2024-01-05', 5),
(6, 3, '2024-01-06', 4),
(7, 3, '2024-01-07', 5),
(8, 4, '2024-01-08', 4),
(9, 4, '2024-01-09', 5),
(10, 4, '2024-01-10', 8),
(11, 4, '2024-01-11', 5);


In [ ]:
-- my sol
with cte1 as
(select o.* , p.product_name as product_name,  p.available_quantity as available_quantity,
COALESCE(sum(quantity_requested) over(partition by o.product_id order by order_date rows between unbounded preceding and 1 preceding),0) as running_qty
from orders o join products p on o.product_id = p.product_id)
-- select * from cte1

select order_id, product_name, quantity_requested,  --available_quantity, running_qty,
case when quantity_requested <= (available_quantity-running_qty) then quantity_requested
    when (available_quantity <= running_qty) then 0
    else (available_quantity-running_qty) end as qty_fulfilled,

case when quantity_requested <= (available_quantity-running_qty) then 'Full Order'
    when (available_quantity <= running_qty) then 'No Order'
    else 'Partial Order' end as comments

from cte1

In [ ]:
-- provided sol
with cte as (
select o.*
,sum(quantity_requested) over(partition by o.product_id order by order_date) as running_requested_qty
,p.available_quantity
,p.product_name
from orders o
inner join products p on o.product_id=p.product_id
)
select order_id,product_name,quantity_requested
,case when  running_requested_qty <= available_quantity then quantity_requested
when available_quantity-(running_requested_qty-quantity_requested) > 0 then available_quantity-(running_requested_qty-quantity_requested)
else 0 end as qty_fulfilled  
,case when  running_requested_qty <= available_quantity then 'Full Order'
when available_quantity-(running_requested_qty-quantity_requested) > 0 then 'Partial Order'
else 'No Order' end as comments  
from cte;

#### [36 - Excess/insufficient Inventory](https://100daysofsql.com/new-courses/1/coding-test/37)

In [ ]:
-- Suppose you are a data analyst working for Flipkart. Your task is to identify excess and insuffienct inventory at various Flipkart warehouses in terms of no of units and cost. 

-- You are given an inventory table containing information about the inventory levels and required inventory target for each product and warehouse location combination.

-- Excess inventory is when inventory levels are greater than inventory targets else its insufficient inventory. Write an SQL to derive excess/insufficient Inventory for each location as well as at overall company level.

-- Table: inventory (primary key : location_id,product_id)

-- column name	datatype
-- location_id	int
-- product_id	int
-- inventory_level	int
-- inventory_target 	int
 

-- Table: products (primary key : product_id)

-- column name	datatype
-- product_id	int
-- unit_cost 	decimal(5,2)
 

-- Expected Output:

-- location_id	excess_insufficient_qty	excess_insufficient_value
-- 1	25	1347.5
-- 2	-25	-1420
-- 3	20	1180
-- 4	-12	-600
-- Overall	8	507.5

In [ ]:
-- Create the inventory table
CREATE TABLE inventory (
    location_id INT,
    product_id INT,
    inventory_level INT,
    inventory_target INT,
    PRIMARY KEY (location_id, product_id)
);

-- Create the products table
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    unit_cost DECIMAL(5, 2)
);

-- Insert data into the inventory table
INSERT INTO inventory (location_id, product_id, inventory_level, inventory_target) VALUES
(1, 101, 90, 80),
(1, 102, 100, 85),
(2, 102, 90, 80),
(2, 103, 70, 95),
(2, 104, 50, 60),
(3, 103, 120, 100),
(4, 104, 90, 102);

-- Insert data into the products table
INSERT INTO products (product_id, unit_cost) VALUES
(101, 51.5),
(102, 55.5),
(103, 59.0),
(104, 50.0);


In [ ]:
-- my sol

with cte as
(select i.*, (inventory_level-inventory_target) as excess_insufficient_qty,
(inventory_level-inventory_target)*p.unit_cost as excess_insufficient_value,
p.unit_cost
from inventory i left join products p on i.product_id=p.product_id)

select
    location_id, sum(excess_insufficient_qty) as excess_insufficient_qty, 
    sum(excess_insufficient_value) as excess_insufficient_value
from cte
group by location_id
union all
select 'Overall' as location_id, sum(excess_insufficient_qty) as excess_insufficient_qty,
sum(excess_insufficient_value) as excess_insufficient_value
from cte

In [ ]:
-- provided solution
with cte as (
select i.location_id
,sum(inventory_level-inventory_target) as excess_insufficient_qty
,sum((inventory_level-inventory_target)*p.unit_cost) as excess_insufficient_value 
from inventory i 
inner join products p on i.product_id=p.product_id
group by i.location_id)
select * from cte
union all
select 'Overall' as location_id 
, sum(excess_insufficient_qty) as excess_insufficient_qty
, sum(excess_insufficient_value) as excess_insufficient_value
from cte;

#### [37 - Zomato Membership](https://100daysofsql.com/new-courses/1/coding-test/38)

In [ ]:
-- Suppose you are working as a data analyst for Zomato . Zomato is planning to offer a premium membership to customers who have placed multiple orders in a single day.

-- Your task is to write a SQL to find those customers who have placed multiple orders in a single day atleast once , total order value generate by those customers and order value generated only by those orders. 

-- Table: orders (primary key : order_id)

-- column name	datatype
-- order_id	int
-- order_date	datetime
-- customer_name	varchar(20)
-- order_value	int
 

-- Expected Output:

-- customer_name	total_order_value	order_value
-- Mudit	780	550
-- Rahul	1300	1150

In [ ]:
-- Create the orders table
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    order_date DATETIME,
    customer_name VARCHAR(20),
    order_value INT
);

-- Insert data into the orders table
INSERT INTO orders (order_id, order_date, customer_name, order_value) VALUES
(1, '2023-01-13 12:30:00', 'Rahul', 250),
(2, '2023-01-13 08:30:00', 'Rahul', 350),
(3, '2023-01-13 09:00:00', 'Mudit', 230),
(4, '2023-01-14 08:30:00', 'Rahul', 150),
(5, '2023-01-14 12:03:00', 'Suresh', 130),
(6, '2023-01-15 09:34:00', 'Mudit', 250),
(7, '2023-01-15 12:30:00', 'Mudit', 300),
(8, '2023-01-15 09:30:00', 'Rahul', 250),
(9, '2023-01-15 12:35:00', 'Rahul', 300),
(10, '2023-01-15 12:03:00', 'Suresh', 130);


In [ ]:
-- my sol
with cte as
(select  distinct customer_name, strftime('%Y%m%d', order_date) as order_day,count(*) as ct
from orders
group by strftime('%Y%m%d', order_date), customer_name
having count(*)>1)

select 
    o.customer_name, sum(order_value) as total_order_value,
    sum(case when ct is not Null then order_value else 0 end ) as order_value
from 
    orders o left join cte on o.customer_name=cte.customer_name 
    and strftime('%Y%m%d', order_date)=order_day
where o.customer_name in (select distinct customer_name from cte)
group by o.customer_name

In [ ]:
-- provided sol
with cte as (
select customer_name,strftime('%Y-%m-%d', order_date) as order_day
,count(*) as no_of_orders
 from orders 
group by customer_name,strftime('%Y-%m-%d', order_date)
having count(*)>1
)
select orders.customer_name,sum(orders.order_value) as total_order_value
,sum(case when cte.customer_name is not null then orders.order_value end) as order_value
from orders
left join cte on orders.customer_name=cte.customer_name and 
strftime('%Y-%m-%d', orders.order_date)=cte.order_day
where orders.customer_name in (select distinct customer_name from cte)
group by orders.customer_name;

#### [38 - Employees Inside Office (Part 1)](https://chatgpt.com/c/e7410c68-202c-4740-a634-92f530e627d0)

In [ ]:
-- A company record its employees movement In and Out of office in a table with 3 columns.

-- Table: employee_record

-- column name	datatype
-- emp_id	int
-- action	varchar(2)
-- created_at	datetime
 

-- Please note below points about the data:

-- 1- First entry for each employee is “in”

-- 2- Every “in” is succeeded by an “out”

-- 3- Employee can work across days

 

-- Write a SQL to find the number of employees inside the Office at “2019-04-01 19:05:00".

In [ ]:
-- Create the employee_record table
CREATE TABLE employee_record (
    emp_id INT,
    action VARCHAR(2),
    created_at DATETIME
);

-- Insert data into the employee_record table
INSERT INTO employee_record (emp_id, action, created_at) VALUES
(1, 'in', '2019-04-01 12:00:00'),
(1, 'out', '2019-04-01 15:00:00'),
(1, 'in', '2019-04-01 17:00:00'),
(1, 'out', '2019-04-01 21:00:00'),
(2, 'in', '2019-04-01 10:00:00'),
(2, 'out', '2019-04-01 16:00:00'),
(3, 'in', '2019-04-01 19:00:00'),
(3, 'out', '2019-04-02 05:00:00'),
(4, 'in', '2019-04-01 10:00:00'),
(4, 'out', '2019-04-01 20:00:00');


#### [39 - Employees Inside Office (Part 2)](https://100daysofsql.com/new-courses/1/coding-test/40)

In [ ]:
-- A company record its employees movement In and Out of office in a table with 3 columns.

-- Table: employee_record

-- column name	datatype
-- emp_id	int
-- action	varchar(2)
-- created_at	datetime
 


-- Please note below points about the data:

-- 1- First entry for each employee is “in”

-- 2- Every “in” is succeeded by an “out”

-- 3- Employee can work across days

 

-- Write an SQL to measure the amount of hours spent by each employee inside the office between “2019-04-01 14:00:00” and “2019-04-02 10:00:00"  in minutes. 

 

-- Expected Output:

-- emp_id	time_spent
-- 1	300.0
-- 2	0.0
-- 3	600.0
-- 4	960.0
-- 5	1200.0
-- 6	0.0

In [ ]:
-- Create the employee_record table
CREATE TABLE employee_record (
    emp_id INT,
    action VARCHAR(2),
    created_at DATETIME
);

-- Insert data into the employee_record table
INSERT INTO employee_record (emp_id, action, created_at) VALUES
(1, 'in', '2019-04-01 12:00:00'),
(1, 'out', '2019-04-01 15:00:00'),
(1, 'in', '2019-04-01 17:00:00'),
(1, 'out', '2019-04-01 21:00:00'),
(2, 'in', '2019-04-01 10:00:00'),
(2, 'out', '2019-04-01 13:00:00'),
(3, 'in', '2019-04-01 19:00:00'),
(3, 'out', '2019-04-02 05:00:00'),
(4, 'in', '2019-04-01 18:00:00'),
(4, 'out', '2019-04-02 20:00:00'),
(5, 'in', '2019-04-01 10:00:00'),
(5, 'out', '2019-04-02 11:00:00'),
(6, 'in', '2019-04-02 11:00:00'),
(6, 'out', '2019-04-02 16:00:00');


In [ ]:
with cte as 
(select * 
, lead(created_at) over(partition by emp_id order by created_at asc) as next_punch
from employee_record)

, cte2 as (select *,
    round((julianday(case when next_punch <'2019-04-01 14:00:00' then 0
        when created_at>'2019-04-02 10:00:00' then 0
        when next_punch<'2019-04-02 10:00:00' then next_punch 
        else '2019-04-02 10:00:00' end) -
    julianday(case when next_punch <'2019-04-01 14:00:00' then 0
        when created_at>'2019-04-02 10:00:00' then 0
        when created_at<'2019-04-01 14:00:00' then '2019-04-01 14:00:00' 
        else created_at end))*(24*60)) as time_spent
from cte
where 
action= 'in')

select 
emp_id, sum(time_spent) as time_spent
from cte2
group by emp_id

#### [40 - Credit Card Transactions (Part 1)](https://100daysofsql.com/new-courses/1/coding-test/41)

In [ ]:
-- You are given a history of credit card transaction data for the people of India across cities as below. Your task is to find out highest spend card type and lowest spent card type for each city.

-- Table: credit_card_transactions (primary key : transaction_id)

-- column name	datatype
-- transaction_id	int
-- transaction_date 	date
-- city 	varchar(20)
-- card_type	varchar(12)
-- gender 	varchar(1)
-- amount 	int
 

-- Expected Output:

-- city	highest_expense_type	lowest_expense_type
-- Bengaluru	Platinum	Silver
-- Delhi	Gold	Platinum
-- Mumbai	Platinum	Gold

In [ ]:
-- Create the credit_card_transactions table
CREATE TABLE credit_card_transactions (
    transaction_id INT PRIMARY KEY,
    transaction_date DATE,
    city VARCHAR(20),
    card_type VARCHAR(12),
    gender VARCHAR(1),
    amount INT
);

-- Insert data into the credit_card_transactions table
INSERT INTO credit_card_transactions (transaction_id, city, transaction_date, card_type, gender, amount) VALUES
(1, 'Delhi', '2024-01-13', 'Gold', 'F', 500),
(2, 'Bengaluru', '2024-01-13', 'Silver', 'M', 1000),
(3, 'Mumbai', '2024-01-14', 'Silver', 'F', 1200),
(4, 'Bengaluru', '2024-01-14', 'Gold', 'M', 900),
(5, 'Bengaluru', '2024-01-14', 'Gold', 'F', 300),
(6, 'Delhi', '2024-01-15', 'Silver', 'M', 200),
(7, 'Mumbai', '2024-01-15', 'Gold', 'F', 900),
(8, 'Delhi', '2024-01-15', 'Gold', 'F', 800),
(9, 'Mumbai', '2024-01-15', 'Silver', 'F', 150),
(10, 'Mumbai', '2024-01-16', 'Platinum', 'F', 1900),
(11, 'Bengaluru', '2024-01-16', 'Platinum', 'M', 1250),
(12, 'Delhi', '2024-01-16', 'Platinum', 'F', 130);


In [ ]:
-- my sol
with cte as 
(select city, card_type, sum(amount) as amount
from credit_card_transactions
group by city, card_type)

,cte2 as 
(select *,
max(amount) over(partition by city) as maxm,
min(amount) over(partition by city) as minm
from cte)

select city,
max(case when amount=maxm then card_type  end) as highest_expense_type,
max(case when amount=minm then card_type end)  as lowest_expense_type
from cte2
group by city

In [ ]:
-- provided sol

with cte as (
select city,card_type,sum(amount) as total_spend 
from credit_card_transactions
group by city,card_type
)
,cte2 as (
select *
,rank() over(partition by city order by total_spend desc) rn_high
,rank() over(partition by city order by total_spend) rn_low
from cte
)
select city
, max(case when rn_high=1 then card_type end) as highest_expense_type
, max(case when rn_low=1 then card_type end) as lowest_expense_type
from cte2
where rn_high=1 or rn_low=1
group by city;

#### [41 - Credit Card Transactions (Part 2)](https://100daysofsql.com/new-courses/1/coding-test/42)

In [ ]:
-- You are given a history of credit card transaction data for the people of India across cities as below. Write an SQL to find percentage contribution of spends by females in each city.  Round the percentage to 2 decimal places.

-- Table: credit_card_transactions (primary key : transaction_id)

-- column name	datatype
-- transaction_id	int
-- transaction_date 	date
-- city 	varchar(20)
-- card_type	varchar(12)
-- gender 	varchar(1)
-- amount 	int

-- Expected Output:

-- city	total_spend	female_spend	female_contribution
-- Bengaluru	3450	300	8.7
-- Delhi	1630	1430	87.73
-- Mumbai	4150	4150	100.0

In [ ]:
-- Create the credit_card_transactions table
CREATE TABLE credit_card_transactions (
    transaction_id INT PRIMARY KEY,
    transaction_date DATE,
    city VARCHAR(20),
    card_type VARCHAR(12),
    gender VARCHAR(1),
    amount INT
);

-- Insert data into the credit_card_transactions table
INSERT INTO credit_card_transactions (transaction_id, city, transaction_date, card_type, gender, amount) VALUES
(1, 'Delhi', '2024-01-13', 'Gold', 'F', 500),
(2, 'Bengaluru', '2024-01-13', 'Silver', 'M', 1000),
(3, 'Mumbai', '2024-01-14', 'Silver', 'F', 1200),
(4, 'Bengaluru', '2024-01-14', 'Gold', 'M', 900),
(5, 'Bengaluru', '2024-01-14', 'Gold', 'F', 300),
(6, 'Delhi', '2024-01-15', 'Silver', 'M', 200),
(7, 'Mumbai', '2024-01-15', 'Gold', 'F', 900),
(8, 'Delhi', '2024-01-15', 'Gold', 'F', 800),
(9, 'Mumbai', '2024-01-15', 'Silver', 'F', 150),
(10, 'Mumbai', '2024-01-16', 'Platinum', 'F', 1900),
(11, 'Bengaluru', '2024-01-16', 'Platinum', 'M', 1250),
(12, 'Delhi', '2024-01-16', 'Platinum', 'F', 130);


In [ ]:
-- # my sol

with cte as 
(select city, gender,
sum(amount) as amount
from credit_card_transactions
group by city, gender )

select city,
sum(amount) as total_spend,
sum(case when gender='F' then amount else 0 end) as female_spend,
round(sum(case when gender='F' then amount else 0 end)*100.0/sum(amount),2) as female_contribution

from cte
group by city

#### [42 - Credit Card Transactions (Part 3)](https://100daysofsql.com/new-courses/1/coding-test/43)

In [ ]:
-- You are given a history of credit card transaction data for the people of India across cities as below. Write an SQL to find how many days each city took to reach cumulative spend of 1500 from its first day of transactions.

-- Table: credit_card_transactions (primary key : transaction_id)

-- column name	datatype
-- transaction_id	int
-- transaction_date 	date
-- city 	varchar(20)
-- card_type	varchar(12)
-- gender 	varchar(1)
-- amount 	int
 

-- Expected Output:

-- city	first_transaction_date	tran_date_1500	no_of_days
-- Bengaluru	2024-01-12	2024-01-14	2.0
-- Delhi	2024-01-13	2024-01-16	3.0
-- Mumbai	2024-01-14	2024-01-18	4.0

In [ ]:
-- Create the credit_card_transactions table
CREATE TABLE credit_card_transactions (
    transaction_id INT PRIMARY KEY,
    transaction_date DATE,
    city VARCHAR(20),
    card_type VARCHAR(12),
    gender VARCHAR(1),
    amount INT
);

-- Insert data into the credit_card_transactions table
INSERT INTO credit_card_transactions (transaction_id, city, transaction_date, card_type, gender, amount) VALUES
(1, 'Delhi', '2024-01-13', 'Gold', 'F', 500),
(2, 'Bengaluru', '2024-01-12', 'Silver', 'M', 1000),
(3, 'Mumbai', '2024-01-14', 'Silver', 'F', 300),
(4, 'Bengaluru', '2024-01-14', 'Gold', 'M', 900),
(5, 'Bengaluru', '2024-01-14', 'Gold', 'F', 300),
(6, 'Delhi', '2024-01-15', 'Silver', 'M', 200),
(7, 'Mumbai', '2024-01-15', 'Gold', 'F', 500),
(8, 'Delhi', '2024-01-15', 'Gold', 'F', 700),
(9, 'Mumbai', '2024-01-15', 'Silver', 'F', 150),
(10, 'Mumbai', '2024-01-18', 'Platinum', 'F', 1000),
(11, 'Bengaluru', '2024-01-16', 'Platinum', 'M', 1250),
(12, 'Delhi', '2024-01-16', 'Platinum', 'F', 130);


In [ ]:
-- # my sol
with cte as 
(select * ,
sum(amount) over(partition by city order by transaction_date asc 
rows between unbounded preceding and current row) as cumm_amt,
min(transaction_date) over(partition by city order by transaction_date asc) as first_transaction

from credit_card_transactions)

-- select * from cte

select city
, first_transaction, transaction_date as tran_date_1500,
julianday(transaction_date)-julianday(first_transaction) as no_of_days
from
(select *,
row_number() over(partition by city order by cumm_amt) as rn
from cte 
where cumm_amt>=1500)
where rn=1